In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

with psycopg2.connect(os.environ['postgre_password_taipei']) as conn:
    with conn.cursor() as cursor:
        #建立資料表
        sql = '''
            create table if not exists 站點資訊(
                站點編號 varchar(10) primary key,
                站點名稱 varchar(30) not null,
                行政區 varchar(10) not null,
                站點地址 varchar(50),
                lat numeric(15,11),
                lng numeric(15,11)
            );

            create table if not exists youbike(
                日期 timestamp,
                編號 varchar(15),
                總車輛 integer,
                可借 integer,
                可還 integer,
                活動 boolean,
                primary key(日期,編號),
                foreign key(編號) references 站點資訊(站點編號)
                on delete cascade
                on update cascade
            );
        '''
        cursor.execute(sql)
print('資料表建立完成')

In [ ]:
import requests
from requests import Response

url:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
r:Response = requests.get(url)
if r.status_code == 200:
    print('download success')

In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()
youbikeData:list[dict] = r.json()

with psycopg2.connect(os.environ['postgre_password_taipei']) as conn:
    with conn.cursor() as cursor:
        #建立資料表
        for item in youbikeData:
            print(item)
            sql = '''
                insert into 站點資訊
                values (%s,%s,%s,%s,%s,%s)
                on conflict do nothing;
            '''
            cursor.execute(sql,(item['sno'],item['sna'],item['sarea'],item['ar'],item['lat'],item['lng']))

            sql = '''
                insert into youbike
                values (%s,%s,%s,%s,%s,%s)
                on conflict do nothing;
                '''
            cursor.execute(sql,(item['mday'],item['sno'],item['tot'],item['sbi'],item['bemp'],item[act]))

            print('資料表插入完成')
            
print('資料全部插入完成')